In [1]:
from pygsti.extras.errorgenpropagation.propagatableerrorgen import *
from pygsti.extras.errorgenpropagation.errorpropagator import *
from pygsti.circuits import Circuit
import numpy as np
import pygsti.processors
import pygsti
import pygsti.tools.lindbladtools as _lt
import scipy
import matplotlib.pyplot as plt

# Introduction to the Propagatable Error Generators Code

## Defining a circuit and error generators

Currently Error Propgagation works for any model that meets three criteria

    1.  The circuit is clifford
    2.  The errors on each gate can be defined at a time t of interest in the small markovian errors basis
    3.  The error error model is defined such that a gate G has some linear combination of error generators following it

We can therefore, start a code by defining a circuit and an error model by simply following the common pyGSTi notation

In [2]:
errorModel={
    'Gxpi2' : {('H','Y'):.01}

}
c=Circuit(10*[('Gxpi2',0)])

Now we can take the above definitions and plug them into the errorpropagator function, to get out a list of post-circuit error generators out.

In [3]:
errors=ErrorPropagator(c,errorModel,BCHOrder=1,BCHLayerwise=False,NonMarkovian=False)

Here BCH order determines the to what order the BCH order will be taken to (if applicable). BCHLayerwise will if false, propagatate all errors to the end before taking the BCH expansion, otherwise it will push the errorgens through a layer and combine with the the error generators for that layer by the rules given by the BCHOrder.  Non-markovian prevents any simplification or BCH expansions being taken, instead allowing the output to be a list a lists, where the each sublist denotes the errorgenerators that were occuring at time t in the circuit.

Additionally, if you want to describe a gate with multiple associated error definitions you can define it as follows.

In [4]:
MultiGateDict={'Gxpi22' : 'Gxpi2'}
errorModel={
    'Gxpi2' : {('H','Y'):.01},
    'Gxpi22' : {('H','X'):.01}

}
c=Circuit(10*[('Gxpi2',0),('Gxpi22',0)])

ErrorPropagator(c,errorModel,MultiGateDict=MultiGateDict, MultiGate=True)

[('H', ('X',), (0.09999999999999999+0j))]

Once the errors are propagated to the process matrix given by the end of circuit error generators is given by

In [6]:
expMat=np.zeros([4**len(c.line_labels),4**len(c.line_labels)],dtype=np.complex128)
for error in errors:
    expMat +=error.toWeightedErrorBasisMatrix()
processMatrix = scipy.linalg.expm(expMat)